# EDA

We will implement a simple EDA from scratch

In [11]:
import random

We can define a simple EDA (PBIL) for Boolean spaces:

In [12]:
class EDA:
    
    def __init__(self, n_vars, pop_size, alpha, b, fit):
        self.n_vars = n_vars 
        self.pop_size = pop_size 
        self.alpha = alpha # How we merge the two distributions 
        self.b = b # What are the fittest individuals that we take (size of truncate)
        self.fit = fit # Fitness function
        self.D = [0.5] * n_vars # The initial distribution is a uniform distribution
        self.pop = [] 
        self.best = None 
        
    # Generation of new solutions according to the marginal distributions D[i]
    # We start with vector 0 and flip the bit accordin to the distribution
    # The probability of flipping the bit is D[i]
    # The result is a binary vector of size n_vars
    def generate_solution(self):
        v = [0] * self.n_vars
        for i in range(0, self.n_vars):
            if random.random() < self.D[i]:
                v[i] = 1
        return v
    
    # Compute the n_vars marginal distributions in the current population
    # We compute the number of 1s for each of the genes and normilize it
    # And we obtain a new set of distributions for each gene
    def compute_dist(self):
        N = [0] * self.n_vars
        for p in self.pop:
            for i in range(0, self.n_vars):
                if p[i] == 1:
                    N[i] += 1
        return [N[i] / self.b for i in range(0, self.n_vars)]
    
    # Update the distributions D[i]
    # We sort by fitness and take the b best individuals
    # We compute the new distributions on b fittest 
    # and update the current distributions according to alpha
    # We also update the best solution found so far
    def update_dist(self):
        self.pop = [self.generate_solution() for _ in range(0, self.pop_size)]
        self.pop.sort(key=self.fit)
        self.pop = self.pop[0:self.b]
        candidate_best = self.pop[0]
        if self.best is None or self.fit(self.best) > self.fit(candidate_best):
            self.best = candidate_best
        N = self.compute_dist()
        for i in range(0, self.n_vars):
            self.D[i] = (1-self.alpha) * self.D[i] + self.alpha * N[i]

We can try the EDA on a simple problem (OneMax), where the only optimum is the vector of all ones

In [13]:
# We want to maximize the number of 1s in the vector
# So, we want to minimize the number of 0s
def onemax(v):
    return v.count(0) 

In [14]:
# 10 bits, 20 individuals, 
# alpha = 0.1 so 90% of the distribution is given by the old one
# b = 5 so we take the best 5 individuals
eda = EDA(10, 20, 0.1, 5, onemax)

We can now look at the distributions $D_i$ after $100$ iterations:

In [ ]:
for i in range(0, 90):
    eda.update_dist()
eda.D

# We found the best solution, and the marginal distribution of 0.999... 
# in each slot is the probability of genereting 1

[0.9998989445386787,
 0.9998712883026933,
 0.9998981752511861,
 0.999850306743446,
 0.999901240656081,
 0.9998986800607592,
 0.9999145338118774,
 0.99986733305359,
 0.9998544943358829,
 0.9998769948759969]

In [18]:
eda = EDA(10, 20, 0.1, 5, onemax)
for i in range(0, 90):
    print(eda.D[0])
    eda.update_dist()

0.5
0.51
0.559
0.5631000000000002
0.5467900000000002
0.5521110000000002
0.5968999000000003
0.5972099100000003
0.6174889190000004
0.6357400271000004
0.6721660243900004
0.6649494219510004
0.6784544797559005
0.6906090317803104
0.7215481286022793
0.7493933157420514
0.7744539841678463
0.7970085857510616
0.8173077271759555
0.81557695445836
0.8340192590125239
0.8506173331112715
0.8655555998001444
0.87900003982013
0.8911000358381169
0.9019900322543052
0.9117910290288747
0.9206119261259872
0.9285507335133885
0.9356956601620496
0.9421260941458447
0.9479134847312602
0.9531221362581342
0.9578099226323208
0.9620289303690887
0.9658260373321798
0.9692434335989618
0.9723190902390656
0.975087181215159
0.9775784630936432
0.9798206167842788
0.9818385551058509
0.9836546995952659
0.9852892296357393
0.9867603066721653
0.9880842760049487
0.9892758484044538
0.9903482635640084
0.9913134372076076
0.9921820934868468
0.9929638841381622
0.993667495724346
0.9943007461519113
0.9948706715367202
0.9953836043830482
0.9

In [16]:
eda.best

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

What is the problem?
If instead of onemax we take a function that is not monotonic,
we can get stuck in a local optimum

In [19]:
# Let's define a more complex fit function
# Now, in this case I want the sime number of 0s and 1s
def f(v):
    return (v.count(0) - v.count(1))**2

In [20]:
eda = EDA(10, 20, 0.1, 5, f)
for i in range(0, 90):
    eda.update_dist()
eda.D

[0.008126359650410682,
 0.4676687489592298,
 0.04352837421625894,
 0.9802879998311946,
 0.25164924647309145,
 0.6751067690603492,
 0.9366207114203833,
 0.5916341342865885,
 0.08666974110711645,
 0.9665536268100764]

In [ ]:
# We again found the best solution, but we have not a good distribution
# We are assuming that each position is indipendent from the others,
# but in this case we have a dependency between the bits
eda.best

[1, 0, 1, 0, 0, 1, 0, 1, 0, 1]